<a href="https://colab.research.google.com/github/rizqinursulistiasari/analisis-sentimen-rohingya/blob/main/analytical_workflow/2_labeling_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Translate Data**

In [ ]:
import pandas as pd

df = pd.read_csv("Rohingya Preprocessing.csv")
df.head(15000)

In [ ]:
#Menghapus data NaN
df_cleaned = df.dropna()
df_cleaned.info()

In [ ]:
# Simpan kembali ke file CSV
df_cleaned.to_csv('Rohingya clean.csv', index=False)

In [ ]:
df = pd.read_csv("Rohingya clean.csv")
df.head(15000)

In [ ]:
#Tahap Awal
data = pd.DataFrame(df['Stemming'])
data.head(15000)

In [ ]:
!pip3 install googletrans==3.1.0a0

In [ ]:
from googletrans import Translator

# Inisialisasi objek Translator
translator = Translator()

# Fungsi untuk menerjemahkan teks
def translate_text(text, target_language='en'):
    translation = translator.translate(text, dest=target_language)
    return translation.text

data['Translate'] = data['Stemming'].apply(translate_text)
data.head(15000)

In [ ]:
def clean_text(text):
    text = text.lower()  # Case folding
    text = re.sub(r'http\S+', '', text)  # Menghapus URL
    text = re.sub(r'@\w+', '', text)  # Menghapus mention
    text = re.sub(r'#\w+', '', text)  # Menghapus hashtag
    text = re.sub(r'[^a-z\s]', '', text)  # Menghapus karakter non-alfabet
    text = re.sub(r'\s+', ' ', text).strip()  # Menghapus spasi berlebih
    return text

data['Translate'] = data['Translate'].apply(clean_text)
data.head(15000)

In [ ]:
data.to_csv('Translate Rohingya.csv', encoding='utf8', index=False)

# **Labelling TextBlob**

In [ ]:
!pip install tweet-preprocessor
!pip install textblob
!pip install wordcloud
!pip install nltk

In [ ]:
import preprocessor as p
from textblob import TextBlob
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
nltk.download('punkt')

In [ ]:
data = pd.read_csv("Rohingya Translate.csv")
data.head(15000)

In [ ]:
# Fungsi untuk melakukan labelling data menggunakan TextBlob
def label_data(text):
    analysis = TextBlob(text)
    if analysis.sentiment.polarity > 0:
        return 'Positif'
    elif analysis.sentiment.polarity < 0:
        return 'Negatif'
    else:
        return 'Netral'

# Melakukan labelling pada setiap data teks
data['Label'] = data['Translate'].apply(label_data)

# Menghitung jumlah data dengan sentimen positif, negatif, dan netral
total_positif = len(data[data['Label'] == 'Positif'])
total_negatif = len(data[data['Label'] == 'Negatif'])
total_netral = len(data[data['Label'] == 'Netral'])
total = len(data)

# Menampilkan hasil analisis data
print(f'Hasil Analisis Data:\nPositif = {total_positif}\nNegatif = {total_negatif}\nNetral = {total_netral}')
print(f'\nTotal Data : {total}')
data.head(15000)

In [ ]:
data['Klasifikasi'] = status
data.head(15000)

In [ ]:
data.to_csv('Rohingya Label TextBlob.csv', encoding='utf8', index=False)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Menghitung frekuensi masing-masing klasifikasi
counts = data['Label'].value_counts()

# Warna untuk setiap kategori
colors = ['#3498db', '#e67e22', '#2ecc71']

# Membuat figure dengan dua subplot berdampingan
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 7))

# Subplot 1: Diagram Batang (Bar Chart)
counts.plot(kind='bar', ax=ax1, color=colors, legend=False)
ax1.set_title('Distribusi Sentimen Publik')
ax1.set_xlabel('Kategori Sentimen')
ax1.set_ylabel('Jumlah')
ax1.set_xticklabels(counts.index, rotation=0)

# Subplot 2: Diagram Lingkaran (Pie Chart)
ax2.pie(counts, labels=None, autopct='%1.1f%%', startangle=140, colors=colors, textprops={'color': 'white', 'fontsize': 15})
ax2.legend(counts.index, loc='upper right')
ax2.axis('equal')  # Pastikan diagram lingkaran menjadi lingkaran
ax2.set_title('Distribusi Sentimen Publik')

# Menampilkan visualisasi
plt.tight_layout()
plt.show()